In [ ]:
# x_1 = 7, x_2 = 6, s_3 = 0, s_4 = 0
obj = [5, 3]
consts = [[[3, 2],2400],[[0, 1],800], [[2, 0],1200]]
#basis [s1, s2]

In [ ]:
def simplexMethod(objFunction: list, constraints: list, operations: list[int], isMax : bool = True) -> list:
    
    """
    Solves the linear programming problem using the Simplex method.

    Parameters:
    @param obj_function (list): The coefficients of the objective function.
    @param constraints (list): A list of constraints, where each constraint is a list containing the coefficients and the right-hand side value.
    @param operations (list): A list of integers representing the type of constraint (1 for <=, 2 for >=, and 3 for =).
    @param is_max (bool): A boolean value indicating whether the objective function is to be maximized or minimized.
    Returns:
    list: The solution to the linear programming problem.
    """

    originalNum = len(constraints)
    internalCount = 0
    originalObj = objFunction.copy()

    # * Add slack variables
    for i in range(originalNum):
        objFunction.append(0)
        for j in range(originalNum):
            if internalCount%(len(constraints)+1) == 0:
                if operations[i] == 1: # <=
                    constraints[j][0].append(1)
                if operations[i] == 2: # >=
                    constraints[j][0].append(-1)
                else: # =
                    constraints[j][0].append(0)
            else:
                constraints[i][0].append(0)
            internalCount += 1
    print
    # * add artificial variables
    artificialVariableIndex = []
    for i in range(len(operations)):
        if operations[i] != 1:
            artificialVariableIndex.append(i)

    print(artificialVariableIndex)
    counter = 0
    checker = False
    for i in range(len(constraints)):
        if i in artificialVariableIndex:
            checker = True
        for j in range(len(artificialVariableIndex)):
            if i in artificialVariableIndex and j == counter and checker:
                constraints[i][0].append(1)
                counter += 1
                checker = False
            else :
                constraints[i][0].append(0)
    for i in range(len(artificialVariableIndex)):
        objFunction.append(0)
    
    for consts in constraints:
        print(consts)

    result = []
    resultIndex = [-1 for i in range(len(constraints))]
    basis = [0 for i in range(len(constraints))]
    z_j = [sum(basis[j]*constraints[j][0][i] for j in range(len(constraints))) for i in range(len(constraints[0][0]))]
    c_j_minus_z_j = [objFunction[i-1] - z_j[i-1] for i in range(len(z_j))]
    while max(c_j_minus_z_j) > 0:  
        maxIndex = c_j_minus_z_j.index(max(c_j_minus_z_j))

        ratios = []
        for i in range(len(constraints)):
            if constraints[i][0][maxIndex] > 0:
                ratios.append((constraints[i][1]/constraints[i][0][maxIndex], i))
        
        if not ratios:
            return "Unbounded solution"
            
        minRatio, minIndex = min(ratios)

        basis[minIndex] = objFunction[maxIndex]
        resultIndex[minIndex] = maxIndex


        pivot = constraints[minIndex][0][maxIndex]

        old_row = constraints[minIndex].copy()

        constraints[minIndex][0] = [x/pivot for x in constraints[minIndex][0]]
        constraints[minIndex][1] = constraints[minIndex][1]/pivot

        for i in range(len(constraints)):
            if i != minIndex:
                multiplier = constraints[i][0][maxIndex]
                constraints[i][0] = [constraints[i][0][j] - multiplier*constraints[minIndex][0][j] 
                                   for j in range(len(constraints[0][0]))]
                constraints[i][1] = constraints[i][1] - multiplier*constraints[minIndex][1]
        
        z_j = [sum(basis[j]*constraints[j][0][i] for j in range(len(basis))) for i in range(len(constraints[0][0]))]
        c_j_minus_z_j = [objFunction[i-1] - z_j[i-1] for i in range(len(z_j))]

    for i in range(len(originalObj)):
        if i in resultIndex:
            result.append(constraints[resultIndex.index(i)][1])
        else:
            result.append(0)
    objFunction = []
    constraints = []
    return result



In [ ]:
resources = [[[2, 3], 1200], [[2, 1], 1000], [[0, 4], 800]]
unitPrice = [3, 4]
operations = [1, 1, 1]
print(simplexMethod(unitPrice, resources, operations))

In [ ]:
profit = [3, 5, 4]
constraintProfit = [[[2, 3, 0],8],[[3, 2, 2], 15],[[0, 4,5], 10]]


In [ ]:
import re

def parseInputConstrict(input_string: str, objVariables: list[str]) -> tuple:
    """
    Parses a linear equation input string and extracts the coefficients and the result.\n
    Args:\n
        input_string (str): The input string representing a linear equation. \n
                            The equation should be in the form of 'ax + by + cz = d' \n
                            where a, b, c, and d are numbers, and x, y, z are variables.\n
    Returns:\n
        tuple: A tuple containing two elements:\n
            - A list of coefficients (float) for each variable in the equation.\n
            - A float representing the result of the equation.
    """

    

    input_string = input_string.replace(" ", "")
    terms = re.findall(r'([+-]?\d*\.?\d*)([a-zA-Z]+)', input_string)
    coefficients = []
    for var in objVariables:
        coefficient = 0
        for term in terms:
            if term[1] == var:
                coefficient = term[0]
                if coefficient in ['+', '-'] or coefficient == '':
                    coefficient += '1'
                coefficient = float(coefficient)
                break
        coefficients.append(coefficient)
    result = input_string.split('=')[-1]
    return [coefficients, float(result)]
equation = "2a + 3b - 4c = 10"
coefficients = parseInputConstrict(equation, ['a', 'e', 'b', 'c'])
print(coefficients)

In [ ]:
def parseInputObj(input_string: str) -> tuple:
    """
    Parses a linear equation input string and extracts the coefficients and the result.\n
    Args:\n
        input_string (str): The input string representing a linear equation. \n
                            The equation should be in the form of 'ax + by + cz = d' \n
                            where a, b, c, and d are numbers, and x, y, z are variables.\n
    Returns:\n
        tuple: A tuple containing two elements:\n
            - A list of coefficients (float) for each variable in the equation.\n
            - A float representing the result of the equation.
    """
    input_string = input_string.replace(" ", "")
    terms = re.findall(r'([+-]?\d*\.?\d*)([a-zA-Z]+)', input_string)
    coefficients = []
    variables = []
    for term in terms:
        coefficient = term[0]
        variable = term[1]
        if coefficient in ['+', '-'] or coefficient == '':
            coefficient += '1'
        coefficients.append(float(coefficient))
        variables.append(variable)
    return coefficients, variables
equation = "20a + 3b - 4c"
coefficients = parseInputObj(equation)
print(coefficients)

In [ ]:
def simplexFunction(obj:list, constrict:list[list,int]) -> list:
    """
    Applies the simplex method to solve a linear programming problem.\n
    Parameters:\n
    obj (list): A list representing the objective function coefficients.\n
    constrict (list[list, int]): A list of constraints, where each constraint is represented 
                                 as a list containing the coefficients and the right-hand side value.\n
    Returns:\n
    list: The solution to the linear programming problem, typically including the optimal values 
          of the decision variables and the optimal value of the objective function.
    """

    obj = parseInputObj(obj)
    return simplexMethod(obj[0], [parseInputConstrict(constraint, obj[1]) for constraint in constrict])

obj = "3a + 5b + 4c + 3d"
constrict = ["2a + 3b + 4d = 10", "3a + 2b + 2c = 15", "a + 4b + 5c = 10"]
print(simplexFunction(obj, constrict))

In [ ]:
def northWestCornerRule(demand, supply, costValueMatrix):
    """ 
    
    1 1 1
    2 2 1
    3 1 1
    
    """

    #DEMAND is the y axis
    #SUPPLY is the x axis

    if sum(demand) != sum(supply):
        return "Invalid input"
    values = []
    keyPairValues = []
    Done = True
    i = 0 
    j = 0
    while Done:
        if supply[i] > demand[j]:
            values.append([demand[j], costValueMatrix[i][j]])
            keyPairValues.append([i, j])
            supply[i] -= demand[j]
            demand[j] = 0
            j += 1
        elif supply[i] < demand[j]:
            values.append([supply[i], costValueMatrix[i][j]])
            keyPairValues.append([i, j])
            demand[j] -= supply[i]
            supply[i] = 0
            i += 1
        elif supply[i] == demand[j]:
            values.append([supply[i], costValueMatrix[i][j]])
            keyPairValues.append([i, j])
            supply[i] = 0
            demand[j] = 0
            Done = False
    results = [value[0]*value[1] for value in values]
    return sum(results), keyPairValues

demand = [10, 20, 30]
supply = [20, 30, 10]
costValueMatrix = [[1, 2, 3], [1, 2, 1], [3, 1, 1]]
values, keyPairValues = northWestCornerRule(demand, supply, costValueMatrix)
print(values, keyPairValues)

In [ ]:
def leastCostRule(demand, supply, costValueMatrix):
    if sum(demand) != sum(supply):
        return "Invalid input"
    adjustedCostValueMatrix = costValueMatrix.copy()
    values = []
    keyPairValues = []
    #m is the demand or y axis
    #n is the supply or x axis
    m = 0
    n = 0
    Done = False
    while not Done:
        minCost = float('inf')
        for row in adjustedCostValueMatrix:
            if min(row) < minCost:
                minCost = min(row)
                m = adjustedCostValueMatrix.index(row)
                n = row.index(min(row))
        print(minCost, m, n)
        if demand[m] > supply[n]:
            values.append([supply[n], minCost])
            keyPairValues.append([m, n])
            demand[m] -= supply[n]
            for i in range(len(demand)):
                adjustedCostValueMatrix[i][n] = float('inf')
            supply[n] = 0

        elif demand[m] < supply[n]:
            values.append([demand[m], minCost])
            keyPairValues.append([m, n])
            supply[n] -= demand[m]
            for i in range(len(supply)):
                adjustedCostValueMatrix[m][i] = float('inf')
            demand[m] = 0
        elif demand[m] == supply[n]:
            values.append([demand[m], minCost])
            keyPairValues.append([m, n])
            demand[m] = 0
            supply[n] = 0
        if sum(demand) == 0 and sum(supply) == 0:
            Done = True
        print(adjustedCostValueMatrix)
    results = [value[0]*value[1] for value in values]
    return sum(results), keyPairValues


demand = [10, 20, 15]
supply = [12, 18, 15]
costValueMatrix = [[2,1,3],[4,5,7],[1,8,6]]
values, keyPairValues = leastCostRule(demand, supply, costValueMatrix)
print(values, keyPairValues)

In [ ]:
def vogelApproximation(costValueMatrix, supply, demand):
    if len(costValueMatrix) != len(demand):
        y = supply.copy()
        x = demand.copy()
    else:
        y = demand.copy()
        x = supply.copy()
    penalty_y = []
    penalty_x = []
    values = []
    keyPairValues = []
    x_index = 0
    y_index = 0
    Done = False
    for row in costValueMatrix:
        penalty_y.append(sorted(row)[1] - sorted(row)[0])
    for i in range(len(costValueMatrix[0])):
        column = [row[i] for row in costValueMatrix]
        penalty_x.append(sorted(column)[1] - sorted(column)[0])
    while not Done:
        if max(penalty_y) > max(penalty_x):
            y_index = penalty_y.index(max(penalty_y))
            minCost = min(costValueMatrix[y_index])
            x_index = costValueMatrix[y_index].index(minCost)
            if x[x_index] > y[y_index]:  # choose y
                values.append([y[y_index], minCost])
                keyPairValues.append([y_index, x_index])
                x[x_index] -= y[y_index]
                for i in range(len(costValueMatrix[0])):
                    costValueMatrix[y_index][i] = float('inf')
                y[y_index] = 0
            elif x[x_index] < y[y_index]:  # choose x
                values.append([x[x_index], minCost])
                keyPairValues.append([y_index, x_index])
                y[y_index] -= x[x_index]
                for i in range(len(costValueMatrix)):
                    costValueMatrix[i][x_index] = float('inf')
                x[x_index] = 0
            elif x[x_index] == y[y_index]:  # choose x
                values.append([x[x_index], minCost])
                keyPairValues.append([y_index, x_index])
                y[y_index] -= x[x_index]
                for i in range(len(costValueMatrix)):
                    costValueMatrix[i][x_index] = float('inf')
                x[x_index] = 0
            penalty_y[y_index] = float('-inf')
            for row in costValueMatrix:
                print(row)
            print(f'-'*10)
        elif max(penalty_y) < max(penalty_x):
            x_index = penalty_x.index(max(penalty_x))
            minCost = float('inf')
            for row in costValueMatrix:
                if min(row) < minCost:
                    minCost = min(row)
                    y_index = costValueMatrix.index(row)
            if x[x_index] > y[y_index]:  # choose y
                values.append([y[y_index], minCost])
                keyPairValues.append([y_index, x_index])
                x[x_index] -= y[y_index]
                for i in range(len(costValueMatrix[0])):
                    costValueMatrix[y_index][i] = float('inf')
                y[y_index] = 0
            elif x[x_index] < y[y_index]:  # choose x
                values.append([x[x_index], minCost])
                keyPairValues.append([y_index, x_index])
                y[y_index] -= x[x_index]
                for i in range(len(costValueMatrix)):
                    costValueMatrix[i][x_index] = float('inf')
                x[x_index] = 0
            elif x[x_index] == y[y_index]:  # choose x
                values.append([x[x_index], minCost])
                keyPairValues.append([y_index, x_index])
                y[y_index] -= x[x_index]
                for i in range(len(costValueMatrix)):
                    costValueMatrix[i][x_index] = float('inf')
                x[x_index] = 0
            penalty_x[x_index] = float('-inf')
            for row in costValueMatrix:
                print(row)
            print(f'-'*10)
        elif max(penalty_y) == max(penalty_x):
            y_index = penalty_y.index(max(penalty_y))
            minCost = min(costValueMatrix[y_index])
            x_index = costValueMatrix[y_index].index(minCost)
            if x[x_index] > y[y_index]:  # choose y
                values.append([y[y_index], minCost])
                keyPairValues.append([y_index, x_index])
                x[x_index] -= y[y_index]
                for i in range(len(costValueMatrix[0])):
                    costValueMatrix[y_index][i] = float('inf')
                y[y_index] = 0
            elif x[x_index] < y[y_index]:  # choose x
                values.append([x[x_index], minCost])
                keyPairValues.append([y_index, x_index])
                y[y_index] -= x[x_index]
                for i in range(len(costValueMatrix)):
                    costValueMatrix[i][x_index] = float('inf')
                x[x_index] = 0
            elif x[x_index] == y[y_index]:  # choose x
                values.append([x[x_index], minCost])
                keyPairValues.append([y_index, x_index])
                y[y_index] -= x[x_index]
                for i in range(len(costValueMatrix)):
                    costValueMatrix[i][x_index] = float('inf')
                x[x_index] = 0
            penalty_y[y_index] = float('-inf')
            for row in costValueMatrix:
                print(row)
            print(f'-'*10)
        if sum(x) <= 0 and sum(y) <= 0:
            Done = True
        for row in costValueMatrix:
            if y[costValueMatrix.index(row)] > 0:
                penalty_y[costValueMatrix.index(row)] = sorted(row)[1] - sorted(row)[0]
        for i in range(len(costValueMatrix[0])):
            if x[i] > 0:
                column = [row[i] for row in costValueMatrix]
                penalty_x[i] = sorted(column)[1] - sorted(column)[0]
    for row in costValueMatrix:
                print(row)
    print(f'-'*10)
    results = [value[0] * value[1] for value in values]
    for values, coordinates in zip(values, keyPairValues):
        print(values, coordinates)
    return sum(results), keyPairValues


demand = [5, 8, 7, 14]
supply = [7, 9, 18]
costValueMatrix = [[19, 30, 50, 10], [70, 30, 40, 60], [40, 8, 70, 20]]
values, keyPairValues = vogelApproximation(costValueMatrix, demand, supply)
print(values, keyPairValues)


In [ ]:
refinery = [[180, 180, float('inf')], [300, 800, 900], [220, 200, 120]]
refineryCopy = [[180, 180, float('inf')], [300, 800, 900], [220, 200, 120]]
demand, supply = [4, 8, 7], [6, 5, 8]
for row in refinery:
    print(row)
print(f'-'*10)
vogels = vogelApproximation(refinery, demand, supply)

print(vogels)
print(f'-'*10)


for vogel in vogels[1]:
    refineryCopy[vogel[0]][vogel[1]] = 0
for row in refineryCopy:
    print(row)
print(f'-'*10)

In [ ]:
sums = vogels[0]
for row in refineryCopy:
    for elements in row:
        if elements != 0 and elements != float('inf') and row.index(elements) != 0:
            sums -= 0.05*elements
print(sums)

In [ ]:
def vogelApproximation(costValueMatrix, supply, demand):
    """
    Implements Vogel's Approximation Method (VAM) to find an initial feasible solution
    for the transportation problem.
    
    Parameters:
    - costValueMatrix (list of lists): The cost matrix where each entry represents
      the transportation cost from a source to a destination.
    - supply (list): The supply available at each source.
    - demand (list): The demand required at each destination.
    
    Returns:
    - totalCost (int): The total transportation cost based on VAM allocation.
    - keyPairValues (list of tuples): The allocation details [(source_index, destination_index)].
    """
    # Check if matrix orientation matches the supply-demand dimensions
    if len(costValueMatrix) != len(demand):
        y = supply.copy()  # Assign supply
        x = demand.copy()  # Assign demand
    else:
        y = demand.copy()
        x = supply.copy()
    
    penalty_y = []  # Row penalties
    penalty_x = []  # Column penalties
    values = []  # Stores allocated quantities and their respective costs
    keyPairValues = []  # Stores (row, column) allocations
    
    x_index = 0  # Column index to be allocated
    y_index = 0  # Row index to be allocated
    Done = False  # Loop termination flag
    
    # Calculate initial penalties for rows (y) and columns (x)
    for row in costValueMatrix:
        penalty_y.append(sorted(row)[1] - sorted(row)[0])  # Difference between two lowest values
    
    for i in range(len(costValueMatrix[0])):
        column = [row[i] for row in costValueMatrix]
        penalty_x.append(sorted(column)[1] - sorted(column)[0])
    
    while not Done:
        # Determine whether row or column has the highest penalty
        if max(penalty_y) > max(penalty_x):
            y_index = penalty_y.index(max(penalty_y))
            minCost = min(costValueMatrix[y_index])
            x_index = costValueMatrix[y_index].index(minCost)
        else:
            x_index = penalty_x.index(max(penalty_x))
            minCost = float('inf')
            for row in costValueMatrix:
                if min(row) < minCost:
                    minCost = min(row)
                    y_index = costValueMatrix.index(row)
        
        # Allocate supply to the selected cell
        if x[x_index] > y[y_index]:  # If supply is greater than demand
            values.append([y[y_index], minCost])
            keyPairValues.append([y_index, x_index])
            x[x_index] -= y[y_index]
            for i in range(len(costValueMatrix[0])):
                costValueMatrix[y_index][i] = float('inf')  # Mark row as processed
            y[y_index] = 0
        elif x[x_index] < y[y_index]:  # If demand is greater than supply
            values.append([x[x_index], minCost])
            keyPairValues.append([y_index, x_index])
            y[y_index] -= x[x_index]
            for i in range(len(costValueMatrix)):
                costValueMatrix[i][x_index] = float('inf')  # Mark column as processed
            x[x_index] = 0
        else:  # If supply equals demand
            values.append([x[x_index], minCost])
            keyPairValues.append([y_index, x_index])
            y[y_index] -= x[x_index]
            for i in range(len(costValueMatrix)):
                costValueMatrix[i][x_index] = float('inf')  # Mark column as processed
            x[x_index] = 0
        
        # Mark processed row/column penalty as negative infinity
        penalty_y[y_index] = float('-inf')
        penalty_x[x_index] = float('-inf')
        
        # Check if transportation is complete
        if sum(x) <= 0 and sum(y) <= 0:
            Done = True
        
        # Recalculate penalties for unprocessed rows and columns
        for row in costValueMatrix:
            if y[costValueMatrix.index(row)] > 0:
                penalty_y[costValueMatrix.index(row)] = sorted(row)[1] - sorted(row)[0]
        for i in range(len(costValueMatrix[0])):
            if x[i] > 0:
                column = [row[i] for row in costValueMatrix]
                penalty_x[i] = sorted(column)[1] - sorted(column)[0]
    
    # Calculate total transportation cost
    results = [value[0] * value[1] for value in values]
    return sum(results), keyPairValues



In [ ]:
def assignmentProblem(matrix):
    # row reduction
    for row in matrix:
        minVal = min(row)
        for i in range(len(row)):
            row[i] -= minVal
    for row in matrix:
        print(row)
    print(f'-'*10)
    # column reduction
    for i in range(len(matrix[0])):
        minVal = min([row[i] for row in matrix])
        for j in range(len(matrix)):
            matrix[j][i] -= minVal
    for row in matrix:
        print(row)
    print(f'-'*10)
    #row scanning
    while True:
        for row in matrix:
            if row.count(0) == 1:
                x = row.index(0)
                y = matrix.index(row)
                for i in range(len(matrix)):
                    if matrix[i][x] == 0 and i != y:
                        matrix[i][x] = float('inf')
        if sum([row.count(0) for row in matrix]) == len(matrix):
            break
    for row in matrix:
        print(row)
    
matrix= [[12,10,10,8],[14,float('inf'),15,11],[6,8,16,4],[8,10,9,7]]
assignmentProblem(matrix)

In [ ]:
import copy
def minimumLines(matrix):
    # row reduction
    for row in matrix:
        minVal = min(row)
        for i in range(len(row)):
            row[i] -= minVal
    for row in matrix:
        print(row)
    print(f'-'*10)
    # column reduction
    for i in range(len(matrix[0])):
        minVal = min([row[i] for row in matrix])
        for j in range(len(matrix)):
            matrix[j][i] -= minVal
    for row in matrix:
        print(row)
    print(f'-'*10)
    #create copy of matrix
    matrixCopy = copy.deepcopy(matrix)
    print("MatrixCopy")
    for row in matrixCopy:
        print(row)
    print(f'-'*10)
    print("Matrix")
    for row in matrix:
        print(row)
    print(f'-'*10)
    #drawing horizontal lines
    for row in matrixCopy:
        if row.count(0) > 1:
            for element in row:
                matrixCopy[matrixCopy.index(row)][row.index(element)] = float('inf')
            pass
    print("MatrixCopy")
    for row in matrixCopy:
        print(row)
    print(f'-'*10)
    print("Matrix")
    for row in matrix:
        print(row)
    print(f'-'*10)
    #drawing vertical lines
    for column in range(len(matrixCopy[0])):
        columnList = [row[column] for row in matrixCopy]
        if columnList.count(0) > 1:
            for row in matrixCopy:
                matrixCopy[matrixCopy.index(row)][column] = float('inf')
    for row in matrixCopy:
        print(row)
    print(f'-'*10)
    #row reduction
    for row in matrixCopy:
        minVal = min(row)
        for i in range(len(row)):
            if row[i] != float('inf'):
                matrix[matrixCopy.index(row)][i] -= minVal
    for row in matrix:
        print(row)

matrix= [[12,10,10,8],[14,float('inf'),15,11],[6,8,16,4],[8,10,9,7]]
minimumLines(matrix)

In [ ]:
import numpy as np

def minimalLines(cost_matrix, show=False):
    def show_matrix(matrix):
        for row in matrix:
            print(row)
        print('-' * 10)
    
    cost_matrix = np.array(cost_matrix)
    rows, cols = cost_matrix.shape
    
    if rows != cols:
        max_size = max(rows, cols)
        new_matrix = np.zeros((max_size, max_size))
        new_matrix[:rows, :cols] = cost_matrix
        cost_matrix = new_matrix
    
    # Step 1: Subtract row minima
    cost_matrix -= cost_matrix.min(axis=1, keepdims=True)
    if show:
        print("After row reduction:")
        show_matrix(cost_matrix)
    
    # Step 2: Subtract column minima
    cost_matrix -= cost_matrix.min(axis=0, keepdims=True)
    if show:
        print("After column reduction:")
        show_matrix(cost_matrix)
    
    # Step 3: Cover all zeros with a minimum number of lines
    def cover_zeros(matrix):
        marked_rows = set()
        marked_cols = set()
        zero_positions = np.where(matrix == 0)
        zero_positions = list(zip(zero_positions[0], zero_positions[1]))
        
        while zero_positions:
            row_counts = np.bincount([r for r, _ in zero_positions], minlength=matrix.shape[0])
            col_counts = np.bincount([c for _, c in zero_positions], minlength=matrix.shape[1])
            
            if max(row_counts) >= max(col_counts):
                row = np.argmax(row_counts)
                marked_rows.add(row)
                zero_positions = [(r, c) for r, c in zero_positions if r != row]
            else:
                col = np.argmax(col_counts)
                marked_cols.add(col)
                zero_positions = [(r, c) for r, c in zero_positions if c != col]
        
        return marked_rows, marked_cols
    
    def adjust_matrix(matrix, marked_rows, marked_cols):
        uncovered_values = [matrix[r, c] for r in range(matrix.shape[0]) for c in range(matrix.shape[1]) 
                            if r not in marked_rows and c not in marked_cols]
        
        min_uncovered_value = min(uncovered_values) if uncovered_values else 0
        matrix[~np.isin(range(matrix.shape[0]), list(marked_rows)), :] -= min_uncovered_value
        matrix[:, np.isin(range(matrix.shape[1]), list(marked_cols))] += min_uncovered_value
    
    while True:
        marked_rows, marked_cols = cover_zeros(cost_matrix)
        covered_line_count = len(marked_rows) + len(marked_cols)
        
        if covered_line_count >= cost_matrix.shape[0]:
            break
        
        adjust_matrix(cost_matrix, marked_rows, marked_cols)
        if show:
            print("Adjusted matrix:")
            show_matrix(cost_matrix)
    
    # Step 4: Assignment
    assigned_positions = []
    covered_rows = set()
    covered_cols = set()
    zero_positions = np.where(cost_matrix == 0)
    zero_positions = list(zip(zero_positions[0], zero_positions[1]))
    
    for row, col in zero_positions:
        if row not in covered_rows and col not in covered_cols:
            assigned_positions.append((row, col))
            covered_rows.add(row)
            covered_cols.add(col)
    
    return assigned_positions

# Example usage
cost_matrix = [
    [37.7, 32.9, 33.8, 37, 35.4],
    [43.4, 33.1, 42.2, 34.7, 41.8],
    [33.3, 28.5, 38.9, 30.4, 33.6],
    [29.2, 26.4, 29.6, 28.5, 31.1]
]

assignments = minimalLines(cost_matrix, show=True)
print("Optimal Assignments:", assignments)


'\nassignments = minimalLines(cost_matrix, show=True)\nprint("Optimal Assignments:", assignments)\n'

### Interpretation of the Answer

The Hungarian algorithm has been applied to solve an assignment problem using the given `cost_matrix`. The results are as follows:

1. **Assignments**:
    - The `assignments` variable contains the optimal assignments as a list of tuples. Each tuple represents a pair `(row, column)` where a task (row) is assigned to a resource (column). 
    - The assignments are: `[(0, 2), (1, 3), (2, 1), (4, 0)]`.

2. **Cost Matrix**:
    - The `cost_matrix` represents the cost of assigning each task to each resource. It is a 4x5 matrix:
      ```
      [[37.7, 32.9, 33.8, 37, 35.4],
        [43.4, 33.1, 42.2, 34.7, 41.8],
        [33.3, 28.5, 38.9, 30.4, 33.6],
        [29.2, 26.4, 29.6, 28.5, 31.1]]
      ```

3. **Swimmers Matrix**:
    - The `swimmers` variable represents the adjusted cost matrix after applying the Hungarian algorithm's steps (row reduction, column reduction, and adjustments). It is a 5x5 matrix:
      ```
      [[3.7, 0.7, 0.0, 3.0, 1.4],
        [8.5, 0.0, 7.3, 0.0, 6.9],
        [2.9, 0.1, 8.5, 0.0, 3.2],
        [0.8, 0.0, 1.2, 0.1, 2.7],
        [0.0, 0.0, 0.0, 0.0, 0.0]]
      ```

4. **Optimal Assignments**:
    - The optimal assignments minimize the total cost of the assignment problem. For example:
      - Backstroke is assigned to David.
      - BreastStroke is assigned to Tony.
      - Butterfly is assigned to Chris.
      - Freestyle is assigned to Carl.to the final assignments.

In [14]:
delhiFlight = [[9, 11], [10, 12], [16, 18], [19, 21]] #0 is departure, 1 is arrival
bombayFlight = [[10, 12], [12, 14], [15, 17], [20, 22]]
matrix = []
row = []
for i in range(len(delhiFlight)):
    for j in range(len(bombayFlight)):
        layOver = bombayFlight[j][0] - (delhiFlight[i][1] + 4)
        if layOver < 0:
            layOver = float('inf')
        row.append(layOver)
    matrix.append(row.copy())
    row.clear()

for row in matrix:
    print(row)

[inf, inf, 0, 5]
[inf, inf, inf, 4]
[inf, inf, inf, inf]
[inf, inf, inf, inf]


to get the layover, i used i used departure of bombay flight - (arrival of delhi + 4h)
if I get an layover < 0, it indicates an impossible flight, thus i indicate it as infinity

So the only possible pairing is 101 to 203, 101 to 204, 102 to 204
The layover is 0, 5, and 4h respectively, thus we must choose either 101 to 203 or 101 to 204

Choosing the lowest layover pairing is 101 to 203 and 102 to 2024